In [1]:
import torch 
torch.cuda.empty_cache()
import unsloth 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🦥 Unsloth Zoo will now patch everything to make training faster!
[2025-03-13 01:24:29,862] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0313 01:24:30.279000 13964 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
from unsloth import FastVisionModel 
import torch 
print(torch.cuda.is_available())
model, tokenizer = FastVisionModel.from_pretrained(
    "./unsloth/Qwen2.5-3B-4bit-pruned",
    load_in_4bit = True , # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = True  , # True or "unsloth" for long context
)

True
==((====))==  Unsloth 2025.3.8: Fast Qwen2_5_Vl vision patching. Transformers: 4.49.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.999 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\utils\modeling.py:784: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

### Load PHOENIX-2014-T dataset

In [4]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict, Video 
import os
from pathlib import Path
import pandas as pd
import gzip 
import pickle

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object



   

def create_phoenix14t_dataset(base_path,dataset_path , eval = False ):
    """
    Create a dataset from Phoenix14T videos
    
    Args:
        base_path: Path to the PHOENIX-2014-T directory containing train/dev/test splits
    """
    list_data_dict = load_dataset_file(dataset_path)
    key_lst = [key for key, value in list_data_dict.items()]
    print("key_lst length ",len(key_lst))
    # Process each split (train, dev, test)
    dataset = []
    for key in key_lst:
        sources = list_data_dict[key]
        video_files = sources["name"]+".mp4"
        video_files = os.path.join(base_path,   video_files)
        data = {
            'video':  str(Path(video_files).resolve()).replace("\\", "/"),
            'video_name': sources["name"], 
            'text': sources["text"]
        }
        dataset.append(data)
    # Convert to DataFrame first (easier to handle)
    df = pd.DataFrame(dataset)
    
    # Convert DataFrame to Dataset
    dataset = Dataset.from_pandas(df)
    return dataset


base_path = "../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px"

train_dataset = create_phoenix14t_dataset(base_path,"data\Phonexi-2014T\labels.train")
dev_dataset = create_phoenix14t_dataset(base_path,"data/Phonexi-2014T/labels.dev")
test_dataset = create_phoenix14t_dataset(base_path,"data/Phonexi-2014T/labels.test")



# Print some information about the dataset
print(f"Train set size: {len(train_dataset)}")
print(f"Dev set size: {len(dev_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Example of accessing a single item
print("\nExample item from training set:")
print(dev_dataset[-1])
sample = dev_dataset[-1]

key_lst length  7096
key_lst length  519
key_lst length  642
Train set size: 7096
Dev set size: 519
Test set size: 642

Example item from training set:
{'video': 'E:/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/27January_2013_Sunday_tagesschau-8836.mp4', 'video_name': 'dev/27January_2013_Sunday_tagesschau-8836', 'text': 'am oberrhein heute nacht bis plus drei grad sonst verbreitet werte um den gefrierpunkt oder leichter frost'}


In [5]:
from unsloth_zoo.vision_utils import get_padding_tokens_ids, _get_dtype, process_vision_info

user_instruction = "Translate the sign language video to German. Pay attention to the facial features, the fingers and the motion of the hands."
system_instruction = "You are a professional German sign language translator."
from decord import VideoReader
def convert_to_conversation(sample, eval = False ):
 

    conversation = [
        { "role": "system",
            "content": [
                {"type": "text", "text": system_instruction}
            ]
        },
        { "role": "user",
            "content": [
                {"type": "text",  "text": user_instruction},
                {"type": "video", "video":sample["video"] }
            ]
        },
        { "role": "assistant",
        
            "content": [
                {"type": "text", "text": sample["text"]}
            ]
        },
    ]
    if eval: 
        conversation = [
        { "role": "system",
            "content": [
                {"type": "text", "text": system_instruction}
            ]
        },
        { "role": "user",
            "content": [
                {"type": "text",  "text": user_instruction},
                {"type": "video", "video":sample["video"] }
            ]
        }
    ]

    return conversation

class SignDataCollator: 
    __slots__ = ("padding_token_ids", "dtype", "ignore_index", "processor", "formatting_func")

    def __init__(self, model, processor,  ignore_index=-100):
        self.padding_token_ids = get_padding_tokens_ids(processor)
        self.dtype = _get_dtype(
            model.config.torch_dtype
            if hasattr(model.config, "torch_dtype")
            else model.get_input_embeddings().weight.dtype
        )
        self.ignore_index = ignore_index
        self.processor = processor
    
    
    def __call__(self, examples): 
        texts = [] 
        videos = [] 

        for example in examples: 
            messages = convert_to_conversation(example, eval = False)
            message= self.processor.apply_chat_template(
                messages,
                tokenize = False,
                add_generation_prompt = False,
            )
            image, video = process_vision_info(messages)
            texts.append(message)
            videos.append(video)

        # Tokenize the texts and process the images
        batch = self.processor(
            text    = texts,
            videos  = videos,
            padding = True,
            # [TODO] Truncating to max_seq_length does NOT work for VLMs
            # truncation = True,
            return_tensors = "pt",
        )

        batch.pop("token_type_ids", None)
        # Pixtral accepts multiple images, so we have to cast it individually
        pixel_values = batch["pixel_values_videos"]
        if type(pixel_values) is list:
            for j, pixel_value_j in enumerate(pixel_values):
                if type(pixel_value_j) is list:
                    for k, pixel_value_k in enumerate(pixel_value_j):
                        pixel_value_j[k] = pixel_value_k.to(self.dtype)
                else:
                    pixel_values[j] = pixel_value_j.to(self.dtype)
            pass
            batch["pixel_values_videos"] = pixel_values
        else:
            batch["pixel_values_videos"] = batch["pixel_values_videos"].to(self.dtype)
        pass

        # Mask image tokens and pad tokens
        labels = batch["input_ids"].clone()
        labels[torch.isin(labels, self.padding_token_ids)] = self.ignore_index
        batch["labels"] = labels
        return batch


### Testing inference 

In [6]:
import torch 
from qwen_vl_utils import process_vision_info
torch.cuda.empty_cache()
FastVisionModel.for_inference(model) # Enable for inference!

print(train_dataset[-1])
messages = convert_to_conversation(train_dataset[-1], eval = False )
# Preparation for inference
print(messages)
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = tokenizer(
    text = [text],
    videos = [video_inputs],
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

{'video': 'E:/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/27January_2013_Sunday_tagesschau-8842.mp4', 'video_name': 'train/27January_2013_Sunday_tagesschau-8842', 'text': 'es bleibt windig'}
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a professional German sign language translator.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Translate the sign language video to German. Pay attention to the facial features, the fingers and the motion of the hands.'}, {'type': 'video', 'video': 'E:/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/27January_2013_Sunday_tagesschau-8842.mp4'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'es bleibt windig'}]}]


qwen-vl-utils using decord to read video.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
from trl import SFTConfig, SFTTrainer
from unsloth import is_bf16_supported
FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = SignDataCollator(model, tokenizer), # Must use!
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
  
        num_train_epochs = 5, # Set this instead of max_steps for full training runs    
        eval_strategy="epoch",
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    ),
)

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,096 | Num Epochs = 5 | Total steps = 4,435
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,084,928/1,818,394,624 (2.26% trained)


ValueError: Video features and video tokens do not match: tokens: 0, features 1001

In [28]:
from unsloth import FastVisionModel 
import torch 
model, tokenizer = FastVisionModel.from_pretrained(
        model_name = "outputs/checkpoint-500", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Set to False for 16bit LoRA
    )
FastVisionModel.for_inference(model) # Enable for inference!

==((====))==  Unsloth 2025.3.8: Fast Qwen2_5_Vl vision patching. Transformers: 4.49.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.999 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2_5_VLForConditionalGeneration(
      (visual): Qwen2_5_VisionTransformerPretrainedModel(
        (patch_embed): Qwen2_5_VisionPatchEmbed(
          (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
        (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
        (blocks): ModuleList(
          (0-31): 32 x Qwen2_5_VLVisionBlock(
            (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
            (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
            (attn): Qwen2_5_VLVisionSdpaAttention(
              (qkv): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1280, out_features=3840, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
                )
                (lora_B): ModuleD

In [29]:
dev_outputs = []
references = [] 
from tqdm import tqdm 
for data in tqdm(dev_dataset): 
    messages = convert_to_conversation(data, eval = True )
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = tokenizer(
        text = [text],
        videos = [video_inputs],
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")
    from transformers import TextStreamer
    # print ("---------------")
    # print (data['text'])
    references.append(data['text'])
    #text_streamer = TextStreamer(tokenizer, skip_prompt = True)
    
    output  = model.generate(**inputs , max_new_tokens = 128) 
    dev_outputs.append(output[0])

final_dev_outputs = tokenizer.batch_decode(dev_outputs, skip_special_tokens = True)





100%|██████████| 519/519 [59:54<00:00,  6.93s/it]   


In [35]:
## save the outputs 
with open("dev_outputs500.txt", "w", encoding = "utf-8") as f:
    for output in final_dev_outputs:
        output = output.split("\nassistant\n")[1].strip()
        f.write(output + "\n")

with open("references500.txt", "w", encoding = "utf-8") as f:
    for reference in references:
        f.write(reference + "\n")

# use sacrebleu to calculate the bleu score 
from sacrebleu import corpus_bleu

bleu_score = corpus_bleu(final_dev_outputs, [references])
print (bleu_score)


BLEU = 1.63 6.4/1.8/1.0/0.6 (BP = 1.000 ratio = 3.834 hyp_len = 26149 ref_len = 6820)


: 

In [17]:
final_dev_outputs = [decoded_output.split("\nassistant\n")[1].strip() for decoded_output in final_dev_outputs]
print(final_dev_outputs)







['dabei strömt kältere luft zu uns', 'das tief über der nordsee lenkt von westen kühle luft nach deutschland', 'im süden schwacher bis mäßiger wind im norden weht er frisch mit starken an der küste auch mit stürmischen böen', 'am samstag ziehen dann von westen wieder neue schneefälle heran die vor allem im süden und osten deutschlands für unwetter sorgen', 'und nun die wettervorhersage für morgen samstag den siebenundzwanzigsten mai', 'im norden und westen ist es meist klar', 'und die temperaturen sinken auf den gefrierpunkt im nordosten und auch im südwesten da bleibt es dann auch noch überwiegend frostfrei', 'heute nacht temperaturen zwischen sieben und ein grad in alpennähe bis minus fünf grad', 'ihnen noch einen schönen abend und machen sie es gut', 'im süden und südosten bleibt es noch trocken und teilweise freundlich', 'im süden und südosten bleibt es meist trocken und teilweise auch freundlich', 'im süden und südwesten ist es heute nacht meist stark bewölkt oder neblig trüb', 'd

In [25]:
for (output, reference) in zip(final_dev_outputs, references):
    # save it in a file
    with open("output.json", "a") as f:
        f.write(output + "\n")
    with open("reference.json", "a") as f:
        f.write(reference + "\n")




In [23]:
from sacrebleu import corpus_bleu
bleu_score = corpus_bleu(final_dev_outputs, [references])
print (bleu_score)


BLEU = 6.21 22.4/6.7/3.7/2.7 (BP = 1.000 ratio = 1.095 hyp_len = 7465 ref_len = 6820)
